<a href="https://colab.research.google.com/github/okahrhan/hands-on-machine-learning-with-scikit-learn_exercises/blob/main/Custom_Models_and_Training_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from tensorflow import keras
import tensorflow as tf

In [8]:
K = keras.backend

In [11]:
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])

In [12]:
K.square(K.transpose(t))+10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

In [15]:
# #example
# model = keras.models.Sequential([
#     keras.layers.Dense(10, activation="relu", input_shape=[8]),
#     keras.layers.Dense(1)
# ])

###Custom Loss Function

In [16]:
def huber_fn(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error)<1
  squared_loss = tf.square(error)/2
  linear_loss = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

In [17]:
#model.compile(loss=huber_fn,opimizer='nadam')
#model.fit(X_train,y_train,[...])


saving and loading models that contain custom components

In [21]:
model = keras.models.load_model('my_model_custom_loss.h5',custom_objects={'huber_fn': huber_fn})


With the current implementation, any error between –1 and 1 is considered “small.”
But what if you want a different threshold? One solution is to create a function that
creates a configured loss function:

In [19]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - (threshold ** 2) / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",custom_objects={"huber_fn": create_huber(2.0)})

Unfortunately, when you save the model, the threshold will not be saved. This means
that you will have to specify the threshold value when loading the model (note that
the name to use is "huber_fn", which is the name of the function you gave Keras, not
the name of the function that created it):

You can solve this by creating a subclass of the keras.losses.Loss class, and then
implementing its get_config() method:

In [ ]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}


You can then use any instance of this class when you compile the model:

In [ ]:
model.compile(loss=HuberLoss(2.), optimizer="nadam")

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5", custom_objects={"HuberLoss": HuberLoss})

###Custom Activation Functions, Initializers, Regularizers and Constrains

In [22]:
def my_softpllus(z):#return value os just tf.nn.softplus(z)
  return tf.math.log(tf.exp(z)+1)

def my_glorot_initilazier(shape, dtype=tf.float32):
  stddev = tf.sqrt(2./(shape[0]+shape[1]))
  return tf.random.normal(shape,stddev=stddev,dtype=dtype)

def my_l1_regularizer(weights):
  return tf.reduce_sum(tf.abs(0.01*weights))

def my_positive_weights(weights):
  return tf.where(weights<0.,tf.zeros_like(weights),weights)



In [ ]:
layer = keras.layers.Dense(30, activation=my_softpllus,kernel_initializer=my_glorot_initilazier,kernel_regularizer=my_l1_regularizer,kernel_constraint=my_positive_weights)

In [ ]:
# #saved taslak
# class MyL1Regularizer(keras.regularizers.Regularizer):
#     def __init__(self, factor):
#         self.factor = factor

#     def __call__(self, weights):
#         return tf.reduce_sum(tf.abs(self.factor * weights))

#     def get_config(self):
#         return {"factor": self.factor}


###Custom Metrics

In [ ]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}


###Custom Layers

In [ ]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))# no weight

In [24]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
      #weight matrix
        self.kernel = self.add_weight(
            name="kernel",
            shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal"
        )
        #bias vector
        self.bias = self.add_weight(
            name="bias",
            shape=[self.units],
            initializer="zeros"
        )
        super().build(batch_input_shape)  # must be at the end
    #forward pass
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    #saved and load
    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "units": self.units,
            "activation": keras.activations.serialize(self.activation),
        }


#example
# model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[28, 28]),
#     MyDense(100, activation="relu"),
#     MyDense(10, activation="softmax")
# ])

###Losses and Metrics Based on Model Internals

In [ ]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [
            keras.layers.Dense(
                30,
                activation="selu",
                kernel_initializer="lecun_normal"
            )
            for _ in range(5)
        ]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)


###Custom Training Loops

In [ ]:
# EPOCHS = 5
# loss_fn = keras.losses.MeanSquaredError()
# optimizer = keras.optimizers.Adam()
# train_loss = keras.metrics.Mean(name="train_loss")
# for epoch in range(EPOCHS):
#     train_loss.reset_state()
#     for step, (x_batch, y_batch) in enumerate(dataset):
#         with tf.GradientTape() as tape:
#             y_pred = model(x_batch, training=True)
#             loss = loss_fn(y_batch, y_pred)

#         grads = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradients(zip(grads, model.trainable_variables))

#         train_loss.update_state(loss)

#     print(f"Epoch {epoch+1}, Loss: {train_loss.result().numpy():.4f}")